<a href="https://colab.research.google.com/github/frank-morales2020/MLxDL/blob/main/RLHF_T2SQL_REPORT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! pip install peft --quiet
! pip install datasets trl ninja packaging --quiet

In [ ]:
# Install Pytorch & other libraries
!pip install torch tensorboard --quiet

# Install Hugging Face libraries
!pip install  --upgrade transformers datasets accelerate evaluate bitsandbytes --quiet

#FlashAttention only supports Ampere GPUs or newer. #NEED A100 IN GOOGLE COLAB
#!pip install -U transformers
!pip install -U flash-attn --no-build-isolation --quiet


! pip install peft --quiet
! pip install datasets trl ninja packaging --quiet

!pip install diffusers safetensors  --quiet
!pip install colab-env --quiet

In [ ]:
!pip install huggingface_hub -q

import colab_env
import os

access_token_write = os.getenv("HUGGINGFACE_ACCESS_TOKEN_WRITE")

In [5]:
#!pip install huggingface_hub --quiet
from huggingface_hub import HfApi

api = HfApi()
api.get_token_permission(token=access_token_write)
#api.set_access_token(access_token)


# frankmorales2020/Mistral-7B-text-to-sql Good

#repo_id = "frankmorales2020/mistral-7b-ppo-poc-t2sql"
#api.create_repo(repo_id=repo_id, private=False)

api.delete_repo(repo_id=repo_id)

#api.upload_folder(
#    folder_path="./model",
#    repo_id=repo_id,
#    repo_type="model",
#)


In [4]:
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    AutoTokenizer,
    TrainingArguments,
)

In [5]:
#frankmorales2020/Mistral-7B-text-to-sql
# Chose the base model you want
#model_name = "frankmorales2020/Mistral-7B-text-to-sql" # 01/03/2023
#frankmorales2020/Mistral-7B-text-to-sql-flash-attention-2
model_name = "frankmorales2020/Mistral-7B-text-to-sql-flash-attention-2" # 04/03/2024
# set device
device = 'cuda'
#v Tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)
# We redefine the pad_token and pad_token_id with out of vocabulary token (unk_token)
tokenizer.pad_token = tokenizer.unk_token
tokenizer.pad_token_id = tokenizer.unk_token_id

tokenizer_config.json:   0%|          | 0.00/1.61k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/51.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/557 [00:00<?, ?B/s]

In [6]:
import colab_env
import os

access_token_write = os.getenv("HUGGINGFACE_ACCESS_TOKEN_WRITE")

from huggingface_hub import login

login(
  token=access_token_write,
  add_to_git_credential=True
)

In [9]:
!pip install huggingface_hub -q

In [10]:
from datasets import load_dataset
from huggingface_hub import hf_hub_url

In [8]:
from datasets import load_dataset

# Convert dataset to OAI messages
system_message = """You are an text to SQL query translator. Users will ask you questions in English and you will generate a SQL query based on the provided SCHEMA.
SCHEMA:
{schema}"""

def create_conversation(sample):
  return {
    "messages": [
      {"role": "system", "content": system_message.format(schema=sample["context"])},
      {"role": "user", "content": sample["question"]},
      {"role": "assistant", "content": sample["answer"]}
    ]
  }

# Load dataset from the hub
dataset = load_dataset("b-mc2/sql-create-context", split="train")
dataset = dataset.shuffle().select(range(12500))

# Convert dataset to OAI messages
dataset = dataset.map(create_conversation, remove_columns=dataset.features,batched=False)

# split dataset into 10,000 training samples and 2,500 test samples
dataset = dataset.train_test_split(test_size=2500/12500)

print(dataset["train"][345]["messages"])

# save datasets to disk
dataset["train"].to_json("train_dataset.json", orient="records")
dataset["test"].to_json("test_dataset.json", orient="records")

Map:   0%|          | 0/12500 [00:00<?, ? examples/s]

[{'content': 'You are an text to SQL query translator. Users will ask you questions in English and you will generate a SQL query based on the provided SCHEMA.\nSCHEMA:\nCREATE TABLE table_18946749_1 (elevation__m_ INTEGER, peak VARCHAR)', 'role': 'system'}, {'content': 'When mount kobowre is the peak what is the highest elevation in meters?', 'role': 'user'}, {'content': 'SELECT MAX(elevation__m_) FROM table_18946749_1 WHERE peak = "Mount Kobowre"', 'role': 'assistant'}]


Creating json from Arrow format:   0%|          | 0/10 [00:00<?, ?ba/s]

Creating json from Arrow format:   0%|          | 0/3 [00:00<?, ?ba/s]

1189478

In [9]:
from datasets import load_dataset
from random import randint

# Load our test dataset and Tokenizer again
#tokenizer = AutoTokenizer.from_pretrained("frankmorales2020/Mistral-7B-text-to-sql") # 01/03/2024
tokenizer = AutoTokenizer.from_pretrained("frankmorales2020/Mistral-7B-text-to-sql-flash-attention-2") # 04/03/2024
eval_dataset = load_dataset("json", data_files="test_dataset.json", split="train")
rand_idx = randint(0, len(eval_dataset))

Generating train split: 0 examples [00:00, ? examples/s]

In [10]:
# generate the same prompt as for the first local test
prompt = tokenizer.apply_chat_template(eval_dataset[rand_idx]["messages"][:2], tokenize=False, add_generation_prompt=True)
request= {"inputs":prompt,"parameters":{"temperature":0.2, "top_p": 0.95, "max_new_tokens": 256}}


Parameter-Efficient Fine-Tuning (PEFT)

In [11]:
import torch
from peft import AutoPeftModelForCausalLM
from transformers import AutoTokenizer, pipeline

peft_model_id = "frankmorales2020/Mistral-7B-text-to-sql-flash-attention-2"


# Load Model with PEFT adapter
model = AutoPeftModelForCausalLM.from_pretrained(
  peft_model_id,
  device_map="auto",
  torch_dtype=torch.float16
)

tokenizer = AutoTokenizer.from_pretrained(peft_model_id)

# load into pipeline
pipe = pipeline("text-generation", model=model, tokenizer=tokenizer)

adapter_config.json:   0%|          | 0.00/710 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.94G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

The new embeddings will be initialized from a multivariate normal distribution that has old embeddings' mean and covariance. As described in this article: https://nlp.stanford.edu/~johnhew/vocab-expansion.html. To disable this, use `mean_resizing=False`
The new lm_head weights will be initialized from a multivariate normal distribution that has old embeddings' mean and covariance. As described in this article: https://nlp.stanford.edu/~johnhew/vocab-expansion.html. To disable this, use `mean_resizing=False`


adapter_model.safetensors:   0%|          | 0.00/1.87G [00:00<?, ?B/s]

The model 'PeftModelForCausalLM' is not supported for text-generation. Supported models are ['BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'LlamaForCausalLM', 'CodeGenForCausalLM', 'CohereForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'DbrxForCausalLM', 'ElectraForCausalLM', 'ErnieForCausalLM', 'FalconForCausalLM', 'FalconMambaForCausalLM', 'FuyuForCausalLM', 'GemmaForCausalLM', 'Gemma2ForCausalLM', 'GitForCausalLM', 'GlmForCausalLM', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoForCausalLM', 'GPTNeoXForCausalLM', 'GPTNeoXJapaneseForCausalLM', 'GPTJForCausalLM', 'GraniteForCausalLM', 'GraniteMoeForCausalLM', 'JambaForCausalLM', 'JetMoeForCausalLM', 'LlamaForCausalLM', 'MambaForCausalLM', 'Mamba2ForCausalLM', 'MarianForCausalLM', 'MBartForCausa

In [12]:
from datasets import load_dataset
from random import randint

# Load our test dataset
eval_dataset = load_dataset("json", data_files="test_dataset.json", split="train")
rand_idx = randint(0, len(eval_dataset))

# Test on sample
prompt = pipe.tokenizer.apply_chat_template(eval_dataset[rand_idx]["messages"][:2], tokenize=False, add_generation_prompt=True)
outputs = pipe(prompt, max_new_tokens=256, do_sample=False, temperature=0.1, top_k=50, top_p=0.1, eos_token_id=pipe.tokenizer.eos_token_id, pad_token_id=pipe.tokenizer.pad_token_id)

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:590: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.1` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:595: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.1` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


In [13]:
print(f"Query:\n{eval_dataset[rand_idx]['messages'][1]['content']}")
print(f"Original Answer:\n{eval_dataset[rand_idx]['messages'][2]['content']}")
print(f"Generated Answer:\n{outputs[0]['generated_text'][len(prompt):].strip()}")

Query:
Who is the opponent of player phil graham?
Original Answer:
SELECT opponent FROM table_name_72 WHERE player = "phil graham"
Generated Answer:
SELECT opponent FROM table_name_72 WHERE player = "phil graham"


https://huggingface.co/frankmorales2020

In [14]:
# Function to get human feedback (replace with your actual implementation)
from transformers import AutoTokenizer, AutoModel
from sklearn.metrics.pairwise import cosine_similarity
import torch

# Load a pre-trained SentenceTransformer model for embedding generation
embedding_model_name = "sentence-transformers/all-mpnet-base-v2"  # Or another suitable model
embedding_model = AutoModel.from_pretrained(embedding_model_name)
embedding_tokenizer = AutoTokenizer.from_pretrained(embedding_model_name)

def get_similarity_score(sentence1, sentence2):
    """
    Calculates the cosine similarity score between the embeddings of two sentences.
    """
    with torch.no_grad():
        embeddings = embedding_model(**embedding_tokenizer(
            [sentence1, sentence2],
            return_tensors="pt",
            padding=True,
            truncation=True
        )).last_hidden_state[:, 0, :]  # Get the [CLS] token embeddings

    similarity = cosine_similarity(embeddings[0].reshape(1, -1), embeddings[1].reshape(1, -1))
    return similarity[0][0]

def get_human_feedback(prompt, generated_answer, expected_answer):
    """
    Automatically approves the generated answer based on similarity score
    compared to the expected answer.
    """

    #print("\nPrompt:", prompt)
    #print("Generated Answer:", generated_answer)
    #print("Expected Answer:", expected_answer)

    # Calculate similarity score
    similarity_score = get_similarity_score(generated_answer, expected_answer)
    print("Similarity Score:", similarity_score)

    # Set a similarity threshold for approval (adjust as needed)
    similarity_threshold = 0.8

    if similarity_score > similarity_threshold:
        return "approve", generated_answer
    else:
        return "correct", expected_answer


config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

In [ ]:
from tqdm import tqdm

# Load our test dataset
eval_dataset = load_dataset("json", data_files="test_dataset.json", split="train")
rand_idx = randint(0, len(eval_dataset))

def evaluate(sample):
    prompt = pipe.tokenizer.apply_chat_template(sample["messages"][:2], tokenize=False, add_generation_prompt=True)
    outputs = pipe(prompt, max_new_tokens=256, do_sample=True, temperature=0.7, top_k=50, top_p=0.95, eos_token_id=pipe.tokenizer.eos_token_id, pad_token_id=pipe.tokenizer.pad_token_id)
    generated_answer = outputs[0]['generated_text'][len(prompt):].strip()
    real_answer = sample["messages"][2]["content"]

    print('\n')
    print('Question: %s\n'%sample["messages"][1]["content"])
    #print(sample["messages"][2]["content"])
    print('\n')
    print('Real Answer: %s'%sample["messages"][2]["content"])
    print('Generated Answer: %s'%outputs[0]['generated_text'][len(prompt):].strip())
    print('\n')

    # Get human feedback
    #get_human_feedback(prompt, generated_answer, expected_answer):
    feedback = get_human_feedback(prompt, generated_answer, real_answer)
    print('\n')
    print('Feedback:', feedback)



    print('\n')
    if  generated_answer == real_answer:
        print('Success!')
        print('Success Answer: %s'%sample["messages"][2]["content"])
        print('\n')
        return 1
    else:
        print('Real Answer: %s'%real_answer)
        print('Failed Answer: %s'%generated_answer)
        print('\n')
        return 0



success_rate = []

number_of_eval_samples = 10
# iterate over eval dataset and predict
for s in tqdm(eval_dataset.shuffle().select(range(number_of_eval_samples))):
    success_rate.append(evaluate(s))

# compute accuracy
accuracy = sum(success_rate)/len(success_rate)

print(f"Accuracy - exact match: {accuracy*100:.2f}%")

When evaluated on 10 samples from the evaluation dataset, our model achieved an impressive accuracy of 90.00%.


In [23]:
from tqdm import tqdm
from difflib import SequenceMatcher
from random import randint
from datasets import load_dataset

# Load our test dataset
eval_dataset = load_dataset("json", data_files="test_dataset.json", split="train")
rand_idx = randint(0, len(eval_dataset))

def evaluate(sample):
    prompt = pipe.tokenizer.apply_chat_template(sample["messages"][:2], tokenize=False, add_generation_prompt=True)
    outputs = pipe(prompt, max_new_tokens=256, do_sample=True, temperature=0.7, top_k=50, top_p=0.95, eos_token_id=pipe.tokenizer.eos_token_id, pad_token_id=pipe.tokenizer.pad_token_id)
    generated_answer = outputs[0]['generated_text'][len(prompt):].strip()
    real_answer = sample["messages"][2]["content"]

    print('\n')
    print('Question: %s\n'%sample["messages"][1]["content"])
    print('\n')
    print('Real Answer: %s'%sample["messages"][2]["content"])
    print('Generated Answer: %s'%outputs[0]['generated_text'][len(prompt):].strip())
    print('\n')

    # Get human feedback (assuming you have a get_human_feedback function)
    feedback = get_human_feedback(prompt, generated_answer, real_answer)
    print('\n')
    print('Feedback:', feedback)

    print('\n')
    if generated_answer == real_answer:
        print('Success!')
        print('Success Answer: %s'%sample["messages"][2]["content"])
        print('\n')
        return 1, generated_answer, prompt  # Return generated_answer and prompt
    else:
        print('Real Answer: %s'%real_answer)
        print('Failed Answer: %s'%generated_answer)
        print('\n')
        return 0, generated_answer, prompt  # Return generated_answer and prompt

def similar(a, b):
    return SequenceMatcher(None, a, b).ratio()

success_rate = []
similarity_scores = []  # List to store similarity scores

number_of_eval_samples = 10
# iterate over eval dataset and predict
for s in tqdm(eval_dataset.shuffle().select(range(number_of_eval_samples))):
    success, generated_answer, prompt = evaluate(s)  # Get generated_answer and prompt
    success_rate.append(success)
    # Calculate and store similarity score
    real_answer = s["messages"][2]["content"]
    similarity_scores.append(similar(real_answer, generated_answer))

# compute accuracy
accuracy = sum(success_rate)/len(success_rate)

# Calculate average similarity
print("Similarity Scores:", similarity_scores)  # Print similarity scores for debugging
avg_similarity = sum(similarity_scores) / len(similarity_scores)

print(f"Accuracy - exact match: {accuracy*100:.2f}%")
print(f"Average Similarity: {avg_similarity*100:.2f}%")

 10%|█         | 1/10 [00:03<00:32,  3.63s/it]



Question: How many appointment dates were recorded when Jürgen Kohler was the replaced by?



Real Answer: SELECT COUNT(date_of_appointment) FROM table_17085981_2 WHERE replaced_by = "Jürgen Kohler"
Generated Answer: SELECT COUNT(date_of_appointment) FROM table_17085981_2 WHERE replaced_by = "Jürgen Kohler"


Similarity Score: 1.0


Feedback: ('approve', 'SELECT COUNT(date_of_appointment) FROM table_17085981_2 WHERE replaced_by = "Jürgen Kohler"')


Success!
Success Answer: SELECT COUNT(date_of_appointment) FROM table_17085981_2 WHERE replaced_by = "Jürgen Kohler"




 20%|██        | 2/10 [00:05<00:22,  2.75s/it]



Question: How many points did Denis Pederson have?



Real Answer: SELECT points FROM table_name_79 WHERE player = "denis pederson"
Generated Answer: SELECT COUNT(points) FROM table_name_79 WHERE player = "denis pederson"


Similarity Score: 0.7711822


Feedback: ('correct', 'SELECT points FROM table_name_79 WHERE player = "denis pederson"')


Real Answer: SELECT points FROM table_name_79 WHERE player = "denis pederson"
Failed Answer: SELECT COUNT(points) FROM table_name_79 WHERE player = "denis pederson"




 30%|███       | 3/10 [00:07<00:16,  2.42s/it]

...



Question: What is listed in tor floysvik when karianne gulliksen is 6?



Real Answer: SELECT MIN(tor_fløysvik) FROM table_28677723_10 WHERE karianne_gulliksen = 6
Generated Answer: SELECT MAX(tor_fløysvik) FROM table_28677723_10 WHERE karianne_gulliksen = 6


Similarity Score: 0.6992097


Feedback: ('correct', 'SELECT MIN(tor_fløysvik) FROM table_28677723_10 WHERE karianne_gulliksen = 6')


Real Answer: SELECT MIN(tor_fløysvik) FROM table_28677723_10 WHERE karianne_gulliksen = 6
Failed Answer: SELECT MAX(tor_fløysvik) FROM table_28677723_10 WHERE karianne_gulliksen = 6




 90%|█████████ | 9/10 [00:24<00:02,  2.87s/it]



Question: What was the year when West Manila has a tariff increase of 6.5?



Real Answer: SELECT year FROM table_17302440_1 WHERE west_manila = "6.5"
Generated Answer: SELECT year FROM table_17302440_1 WHERE west_manila = "Tariff increase of 6.5"


Similarity Score: 0.8412812


Feedback: ('approve', 'SELECT year FROM table_17302440_1 WHERE west_manila = "Tariff increase of 6.5"')


Real Answer: SELECT year FROM table_17302440_1 WHERE west_manila = "6.5"
Failed Answer: SELECT year FROM table_17302440_1 WHERE west_manila = "Tariff increase of 6.5"




100%|██████████| 10/10 [00:26<00:00,  2.64s/it]



Question: What is the score when the player is Matt Kuchar?



Real Answer: SELECT score FROM table_28498999_5 WHERE player = "Matt Kuchar"
Generated Answer: SELECT score FROM table_28498999_5 WHERE player = "Matt Kuchar"


Similarity Score: 1.0000002


Feedback: ('approve', 'SELECT score FROM table_28498999_5 WHERE player = "Matt Kuchar"')


Success!
Success Answer: SELECT score FROM table_28498999_5 WHERE player = "Matt Kuchar"


Similarity Scores: [1.0, 0.9481481481481482, 1.0, 1.0, 1.0, 1.0, 0.9428571428571428, 0.9736842105263158, 0.8613138686131386, 1.0]
Accuracy - exact match: 60.00%
Average Similarity: 97.26%


In [25]:
from tqdm import tqdm
from difflib import SequenceMatcher
from random import randint
from datasets import load_dataset

# Load our test dataset
eval_dataset = load_dataset("json", data_files="test_dataset.json", split="train")
rand_idx = randint(0, len(eval_dataset))

def evaluate(sample):
    prompt = pipe.tokenizer.apply_chat_template(sample["messages"][:2], tokenize=False, add_generation_prompt=True)
    outputs = pipe(prompt, max_new_tokens=256, do_sample=True, temperature=0.7, top_k=50, top_p=0.95, eos_token_id=pipe.tokenizer.eos_token_id, pad_token_id=pipe.tokenizer.pad_token_id)
    generated_answer = outputs[0]['generated_text'][len(prompt):].strip()
    real_answer = sample["messages"][2]["content"]

    print('\n')
    print('Question: %s\n'%sample["messages"][1]["content"])
    print('\n')
    print('Real Answer: %s'%sample["messages"][2]["content"])
    print('Generated Answer: %s'%outputs[0]['generated_text'][len(prompt):].strip())
    print('\n')

    # Get human feedback (this is where you'll use RLHF feedback)
    feedback = get_human_feedback(prompt, generated_answer, real_answer)
    print('\n')
    print('Feedback:', feedback)

    print('\n')
    if generated_answer == real_answer:
        print('Success!')
        print('Success Answer: %s'%sample["messages"][2]["content"])
        print('\n')
        return 1, generated_answer, prompt, feedback  # Return feedback as well
    else:
        print('Real Answer: %s'%real_answer)
        print('Failed Answer: %s'%generated_answer)
        print('\n')
        return 0, generated_answer, prompt, feedback  # Return feedback as well

def similar(a, b):
    return SequenceMatcher(None, a, b).ratio()

def calculate_rlhf_score(feedback):
    # Implement your logic to convert RLHF feedback to a score
    if feedback[0] == "approve":  # Assuming "approve" means correct and good
        return 1.0
    elif feedback[0] == "fix":  # Assuming "fix" means needs improvement
        return 0.5  # You can adjust this score based on how much improvement is needed
    else:
        return 0.0  # For other feedback types, you might assign different scores

success_rate = []
similarity_scores = []
rlhf_scores = []  # List to store RLHF scores

number_of_eval_samples = 10

for s in tqdm(eval_dataset.shuffle().select(range(number_of_eval_samples))):
    success, generated_answer, prompt, feedback = evaluate(s)  # Get feedback
    success_rate.append(success)
    real_answer = s["messages"][2]["content"]
    similarity_scores.append(similar(real_answer, generated_answer))
    rlhf_scores.append(calculate_rlhf_score(feedback))  # Calculate and store RLHF score

accuracy = sum(success_rate) / len(success_rate)

print("Similarity Scores:", similarity_scores)
avg_similarity = sum(similarity_scores) / len(similarity_scores)

print("RLHF Scores:", rlhf_scores)  # Print RLHF scores
avg_rlhf_score = sum(rlhf_scores) / len(rlhf_scores)  # Calculate average RLHF score

print(f"Accuracy - exact match: {accuracy*100:.2f}%")
print(f"Average Similarity: {avg_similarity*100:.2f}%")
print(f"Average RLHF Score: {avg_rlhf_score*100:.2f}%")

 10%|█         | 1/10 [00:02<00:24,  2.75s/it]



Question: What is the sum of Poles, when Season is greater than 2004, and when Podiums is less than 1?



Real Answer: SELECT SUM(poles) FROM table_name_64 WHERE season < 2004 AND podiums < 1
Generated Answer: SELECT SUM(poles) FROM table_name_64 WHERE season > 2004 AND podiums < 1


Similarity Score: 0.997982


Feedback: ('approve', 'SELECT SUM(poles) FROM table_name_64 WHERE season > 2004 AND podiums < 1')


Real Answer: SELECT SUM(poles) FROM table_name_64 WHERE season < 2004 AND podiums < 1
Failed Answer: SELECT SUM(poles) FROM table_name_64 WHERE season > 2004 AND podiums < 1




 20%|██        | 2/10 [00:04<00:15,  1.98s/it]



Question: What are the names of wrestlers days held less than 100?



Real Answer: SELECT Name FROM wrestler WHERE Days_held < 100
Generated Answer: SELECT Name FROM wrestler WHERE Days_held < 100


Similarity Score: 1.0


Feedback: ('approve', 'SELECT Name FROM wrestler WHERE Days_held < 100')


Success!
Success Answer: SELECT Name FROM wrestler WHERE Days_held < 100




 30%|███       | 3/10 [00:06<00:15,  2.21s/it]

...



Question: What is the high grid for osella - alfa romeo, and a Laps larger than 61?



Real Answer: SELECT MAX(grid) FROM table_name_56 WHERE constructor = "osella - alfa romeo" AND laps > 61
Generated Answer: SELECT MAX(grid) FROM table_name_56 WHERE constructor = "osella - alfa romeo" AND laps > 61


Similarity Score: 0.9999999


Feedback: ('approve', 'SELECT MAX(grid) FROM table_name_56 WHERE constructor = "osella - alfa romeo" AND laps > 61')


Success!
Success Answer: SELECT MAX(grid) FROM table_name_56 WHERE constructor = "osella - alfa romeo" AND laps > 61




 90%|█████████ | 9/10 [00:21<00:02,  2.30s/it]



Question: Find all the distinct district names ordered by city area in descending.



Real Answer: SELECT DISTINCT District_name FROM district ORDER BY city_area DESC
Generated Answer: SELECT District_name FROM district ORDER BY city_area DESC


Similarity Score: 0.9165655


Feedback: ('approve', 'SELECT District_name FROM district ORDER BY city_area DESC')


Real Answer: SELECT DISTINCT District_name FROM district ORDER BY city_area DESC
Failed Answer: SELECT District_name FROM district ORDER BY city_area DESC




100%|██████████| 10/10 [00:24<00:00,  2.46s/it]



Question: What's the lane number when time was 20.45 and the react was less than 0.164?



Real Answer: SELECT MAX(lane) FROM table_name_85 WHERE react < 0.164 AND time = "20.45"
Generated Answer: SELECT SUM(lane) FROM table_name_85 WHERE react < 0.164 AND time = 20.45


Similarity Score: 0.73968095


Feedback: ('correct', 'SELECT MAX(lane) FROM table_name_85 WHERE react < 0.164 AND time = "20.45"')


Real Answer: SELECT MAX(lane) FROM table_name_85 WHERE react < 0.164 AND time = "20.45"
Failed Answer: SELECT SUM(lane) FROM table_name_85 WHERE react < 0.164 AND time = 20.45


Similarity Scores: [0.9861111111111112, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.928, 0.958904109589041]
RLHF Scores: [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0]
Accuracy - exact match: 70.00%
Average Similarity: 98.73%
Average RLHF Score: 90.00%


## VERSION - FINAL

In [28]:
from tqdm import tqdm
from difflib import SequenceMatcher
from random import randint
from datasets import load_dataset
import sqlparse  # Import sqlparse for SQL parsing

# Load our test dataset
eval_dataset = load_dataset("json", data_files="test_dataset.json", split="train")

def evaluate(sample):
    prompt = pipe.tokenizer.apply_chat_template(sample["messages"][:2], tokenize=False, add_generation_prompt=True)
    outputs = pipe(prompt, max_new_tokens=256, do_sample=True, temperature=0.7, top_k=50, top_p=0.95, eos_token_id=pipe.tokenizer.eos_token_id, pad_token_id=pipe.tokenizer.pad_token_id)
    generated_answer = outputs[0]['generated_text'][len(prompt):].strip()
    real_answer = sample["messages"][2]["content"]

    print('\n')
    print('Question: %s\n'%sample["messages"][1]["content"])
    print('Real Answer: %s'%real_answer)
    print('Generated Answer: %s'%generated_answer)
    print('\n')

    # Get human feedback
    feedback = get_human_feedback(prompt, generated_answer, real_answer)
    print('Feedback:', feedback)
    print('\n')

    # Normalize SQL queries before comparison (using sqlparse)
    real_answer_normalized = sqlparse.format(real_answer, reindent=True, keyword_case='upper')
    generated_answer_normalized = sqlparse.format(generated_answer, reindent=True, keyword_case='upper')

    if generated_answer_normalized == real_answer_normalized:
        print('Success!')
        print('Success Answer: %s'%real_answer)
        print('\n')
        return 1, generated_answer, prompt, feedback
    else:
        print('Real Answer: %s'%real_answer)
        print('Failed Answer: %s'%generated_answer)
        print('\n')
        return 0, generated_answer, prompt, feedback

def similar(a, b):
    return SequenceMatcher(None, a, b).ratio()

def calculate_rlhf_score(feedback):
    if feedback[0] == "approve":
        return 1.0
    elif feedback[0] == "fix":
        return 0.5
    else:
        return 0.0

success_rate = []
similarity_scores = []
rlhf_scores = []

number_of_eval_samples = 10

for s in tqdm(eval_dataset.shuffle().select(range(number_of_eval_samples))):
    success, generated_answer, prompt, feedback = evaluate(s)
    success_rate.append(success)
    real_answer = s["messages"][2]["content"]

    # Normalize SQL queries before calculating similarity
    real_answer_normalized = sqlparse.format(real_answer, reindent=True, keyword_case='upper')
    generated_answer_normalized = sqlparse.format(generated_answer, reindent=True, keyword_case='upper')

    similarity_scores.append(similar(real_answer_normalized, generated_answer_normalized))
    rlhf_scores.append(calculate_rlhf_score(feedback))

accuracy = sum(success_rate) / len(success_rate)

print("Similarity Scores:", similarity_scores)
avg_similarity = sum(similarity_scores) / len(similarity_scores)

print("RLHF Scores:", rlhf_scores)
avg_rlhf_score = sum(rlhf_scores) / len(rlhf_scores)

print(f"Accuracy - exact match: {accuracy*100:.2f}%")
print(f"Average Similarity: {avg_similarity*100:.2f}%")
print(f"Average RLHF Score: {avg_rlhf_score*100:.2f}%")

 10%|█         | 1/10 [00:05<00:48,  5.38s/it]



Question: Who were the starting actors in the time frame of  tuesday 22:00~22:54 8 april 2008 to 17 june 2008?

Real Answer: SELECT starring_actors FROM table_18539834_2 WHERE time_frame = "Tuesday 22:00~22:54 8 April 2008 to 17 June 2008"
Generated Answer: SELECT starring_actors FROM table_18539834_2 WHERE time_frame = "Tuesday 22:00~22:54 8 April 2008 to 17 June 2008"


Similarity Score: 1.0
Feedback: ('approve', 'SELECT starring_actors FROM table_18539834_2 WHERE time_frame = "Tuesday 22:00~22:54 8 April 2008 to 17 June 2008"')


Success!
Success Answer: SELECT starring_actors FROM table_18539834_2 WHERE time_frame = "Tuesday 22:00~22:54 8 April 2008 to 17 June 2008"




 20%|██        | 2/10 [00:07<00:28,  3.59s/it]



Question: What was the highest attendance at a game that was played in tulane stadium?

Real Answer: SELECT MAX(attendance) FROM table_name_63 WHERE game_site = "tulane stadium"
Generated Answer: SELECT MAX(attendance) FROM table_name_63 WHERE game_site = "tulane stadium"


Similarity Score: 1.0000001
Feedback: ('approve', 'SELECT MAX(attendance) FROM table_name_63 WHERE game_site = "tulane stadium"')


Success!
Success Answer: SELECT MAX(attendance) FROM table_name_63 WHERE game_site = "tulane stadium"




 30%|███       | 3/10 [00:09<00:20,  2.87s/it]

...



Question: What stadium is in Seoul?

Real Answer: SELECT stadium FROM table_name_56 WHERE city = "seoul"
Generated Answer: SELECT stadium FROM table_name_56 WHERE city = "seoul"


Similarity Score: 1.0
Feedback: ('approve', 'SELECT stadium FROM table_name_56 WHERE city = "seoul"')


Success!
Success Answer: SELECT stadium FROM table_name_56 WHERE city = "seoul"




 90%|█████████ | 9/10 [00:22<00:02,  2.23s/it]



Question: How many total drawn has less than 4 lost and 8 points?

Real Answer: SELECT SUM(drawn) FROM table_name_59 WHERE lost < 4 AND points = 8
Generated Answer: SELECT SUM(drawn) FROM table_name_59 WHERE lost < 4 AND points = 8


Similarity Score: 1.0000001
Feedback: ('approve', 'SELECT SUM(drawn) FROM table_name_59 WHERE lost < 4 AND points = 8')


Success!
Success Answer: SELECT SUM(drawn) FROM table_name_59 WHERE lost < 4 AND points = 8




100%|██████████| 10/10 [00:24<00:00,  2.49s/it]



Question: Which FA Cup Apps have League Apps of 21?

Real Answer: SELECT fa_cup_apps FROM table_name_28 WHERE league_apps = "21"
Generated Answer: SELECT fa_cup_apps FROM table_name_28 WHERE league_apps = "21"


Similarity Score: 1.0000005
Feedback: ('approve', 'SELECT fa_cup_apps FROM table_name_28 WHERE league_apps = "21"')


Success!
Success Answer: SELECT fa_cup_apps FROM table_name_28 WHERE league_apps = "21"


Similarity Scores: [1.0, 1.0, 0.9343065693430657, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]
RLHF Scores: [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]
Accuracy - exact match: 90.00%
Average Similarity: 99.34%
Average RLHF Score: 100.00%


In [29]:
from tqdm import tqdm
from difflib import SequenceMatcher
from random import randint
from datasets import load_dataset
import sqlparse  # Import sqlparse for SQL parsing

# Load our test dataset
eval_dataset = load_dataset("json", data_files="test_dataset.json", split="train")

def evaluate(sample):
    prompt = pipe.tokenizer.apply_chat_template(sample["messages"][:2], tokenize=False, add_generation_prompt=True)
    outputs = pipe(prompt, max_new_tokens=256, do_sample=True, temperature=0.7, top_k=50, top_p=0.95, eos_token_id=pipe.tokenizer.eos_token_id, pad_token_id=pipe.tokenizer.pad_token_id)
    generated_answer = outputs[0]['generated_text'][len(prompt):].strip()
    real_answer = sample["messages"][2]["content"]

    print('\n')
    print('Question: %s\n'%sample["messages"][1]["content"])
    print('Real Answer: %s'%real_answer)
    print('Generated Answer: %s'%generated_answer)
    print('\n')

    # Get human feedback
    feedback = get_human_feedback(prompt, generated_answer, real_answer)
    print('Feedback:', feedback)
    print('\n')

    # Normalize SQL queries before comparison (using sqlparse)
    real_answer_normalized = sqlparse.format(real_answer, reindent=True, keyword_case='upper')
    generated_answer_normalized = sqlparse.format(generated_answer, reindent=True, keyword_case='upper')

    if generated_answer_normalized == real_answer_normalized:
        print('Success!')
        print('Success Answer: %s'%real_answer)
        print('\n')
        return 1, generated_answer, prompt, feedback
    else:
        print('Real Answer: %s'%real_answer)
        print('Failed Answer: %s'%generated_answer)
        print('\n')
        return 0, generated_answer, prompt, feedback

def similar(a, b):
    return SequenceMatcher(None, a, b).ratio()

def calculate_rlhf_score(feedback):
    if feedback[0] == "approve":
        return 1.0
    elif feedback[0] == "fix":
        return 0.5
    else:
        return 0.0

success_rate = []
similarity_scores = []
rlhf_scores = []

number_of_eval_samples = 1000

for s in tqdm(eval_dataset.shuffle().select(range(number_of_eval_samples))):
    success, generated_answer, prompt, feedback = evaluate(s)
    success_rate.append(success)
    real_answer = s["messages"][2]["content"]

    # Normalize SQL queries before calculating similarity
    real_answer_normalized = sqlparse.format(real_answer, reindent=True, keyword_case='upper')
    generated_answer_normalized = sqlparse.format(generated_answer, reindent=True, keyword_case='upper')

    similarity_scores.append(similar(real_answer_normalized, generated_answer_normalized))
    rlhf_scores.append(calculate_rlhf_score(feedback))

accuracy = sum(success_rate) / len(success_rate)

print("Similarity Scores:", similarity_scores)
avg_similarity = sum(similarity_scores) / len(similarity_scores)

print("RLHF Scores:", rlhf_scores)
avg_rlhf_score = sum(rlhf_scores) / len(rlhf_scores)

print(f"Accuracy - exact match: {accuracy*100:.2f}%")
print(f"Average Similarity: {avg_similarity*100:.2f}%")
print(f"Average RLHF Score: {avg_rlhf_score*100:.2f}%")

  0%|          | 1/1000 [00:01<27:27,  1.65s/it]



Question: Which city has an opponent of England?

Real Answer: SELECT city FROM table_name_99 WHERE opponent = "england"
Generated Answer: SELECT city FROM table_name_99 WHERE opponent = "england"


Similarity Score: 1.0
Feedback: ('approve', 'SELECT city FROM table_name_99 WHERE opponent = "england"')


Success!
Success Answer: SELECT city FROM table_name_99 WHERE opponent = "england"




  0%|          | 2/1000 [00:03<27:12,  1.64s/it]



Question: On what platform does Mindark publish?

Real Answer: SELECT platform FROM table_name_84 WHERE publisher = "mindark"
Generated Answer: SELECT platform FROM table_name_84 WHERE publisher = "mindark"


Similarity Score: 1.0
Feedback: ('approve', 'SELECT platform FROM table_name_84 WHERE publisher = "mindark"')


Success!
Success Answer: SELECT platform FROM table_name_84 WHERE publisher = "mindark"




  0%|          | 3/1000 [00:06<35:56,  2.16s/it]

...



Question: What is Player, when To Par is less than 14?

Real Answer: SELECT player FROM table_name_28 WHERE to_par < 14
Generated Answer: SELECT player FROM table_name_28 WHERE to_par < 14


Similarity Score: 0.9999998
Feedback: ('approve', 'SELECT player FROM table_name_28 WHERE to_par < 14')


Success!
Success Answer: SELECT player FROM table_name_28 WHERE to_par < 14




100%|█████████▉| 999/1000 [44:02<00:01,  1.96s/it]



Question: Can you tell me the Chassis that has the Year of 1977?

Real Answer: SELECT chassis FROM table_name_47 WHERE year = 1977
Generated Answer: SELECT chassis FROM table_name_47 WHERE year = "1977"


Similarity Score: 0.99522567
Feedback: ('approve', 'SELECT chassis FROM table_name_47 WHERE year = "1977"')


Real Answer: SELECT chassis FROM table_name_47 WHERE year = 1977
Failed Answer: SELECT chassis FROM table_name_47 WHERE year = "1977"




100%|██████████| 1000/1000 [44:04<00:00,  2.64s/it]



Question: What's the length of the model with 500 E body style?

Real Answer: SELECT length FROM table_1773707_2 WHERE body_style = "500 E"
Generated Answer: SELECT length FROM table_1773707_2 WHERE body_style = "500 E"


Similarity Score: 1.0
Feedback: ('approve', 'SELECT length FROM table_1773707_2 WHERE body_style = "500 E"')


Success!
Success Answer: SELECT length FROM table_1773707_2 WHERE body_style = "500 E"


Similarity Scores: [1.0, 1.0, 0.9883720930232558, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.9672131147540983, 1.0, 1.0, 1.0, 1.0, 1.0, 0.9875, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.9733333333333334, 1.0, 1.0, 1.0, 1.0, 0.9811320754716981, 1.0, 0.8321167883211679, 0.9496402877697842, 1.0, 1.0, 1.0, 0.9558823529411765, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.950354609929078, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.9559748427672956, 1.0, 0.9692307692307692, 1.0, 0.987012987012987, 1.0, 1.0, 1.0, 1.0, 0.9803921568627451, 1.0, 1.0, 1.0, 1.0, 